In [1]:
import json
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
%matplotlib inline

In [20]:
with open('cook_dataset.json') as json_file:  
    data = json.load(json_file)

In [90]:
df = pd.DataFrame(data)
df.head()
df.loc[7]

cuisine                                                 russkaya
ingredients    [Куриное филе, Свежие шампиньоны, Лук репчатый...
name                             Жюльен с курицей и шампиньонами
rating                                                      2863
Name: 7, dtype: object

In [4]:
len(df['cuisine'].unique())

14

In [7]:
ingr = df['ingredients']
set_of_ingr = set()
for i in ingr:
    set_of_ingr.update(i)

2301

In [ ]:
for i in set_of_ingr:
    df.loc[:,i] = pd.Series(np.zeros(df.shape[0]),index=df.index)
df.head()

In [ ]:
df.shape

In [ ]:
for index, row in df.iterrows():
    for j in set_of_ingr:
        if j in df.ingredients[index]:
            df[j][index] = 1

In [ ]:
with open('df_classif.pickle', 'wb') as f:
    pickle.dump(df, f)

Перебор по параметрам

In [6]:
parameters = {'solver':('newton-cg', 'lbfgs',), 'C':[1,10,100]}
grid = GridSearchCV(LogisticRegression(), cv=5, n_jobs=1, param_grid=parameters)
grid.fit(X_train, y_train)
grid.best_params_

{'C': 1, 'solver': 'newton-cg'}

In [8]:
grid.score(X_test,y_test)

0.5057283142389526

accuracy

In [2]:
with open('df_classif.pickle', 'rb') as f:
    df = pickle.load(f)
df = df.loc[~df['cuisine'].isin(['armyanskaya','azerbaydzhanskaya','tayskaya','uzbekskaya'])]
y = df['cuisine']
df1 = df.drop(['cuisine','ingredients', 'name', 'rating'],axis=1)

In [5]:
df1.head()

,Блины,Баранья грудинка,Крутоны,Чай аньцзи байча,Грог масала,Апельсиновый ароматизатор,Финики,Соус демигляс,Филе палтуса,Черные помидоры,...,Мелкие патиссоны,Желтый карри порошок,Сушеные помидоры,Говяжья пашина,Турнепс,Мясо,Сушеный перец чили,Красный перец чили,Шоколадные пряники,Яблоки «голден»
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1.values, y, test_size=0.1, random_state=42)

In [4]:
%%time
clf = LogisticRegression(solver='newton-cg', multi_class='multinomial',n_jobs=1).fit(X_train, y_train)

Wall time: 6.16 s


In [5]:
def byVal_key(kort):
    return kort[0]

In [6]:
def score_comp(X,y,clf,num_of_cl=3):
    y_pred = list(clf.predict_proba(X))
    
    for i in range(len(y_pred)):
        y_pred[i] = list(zip(y_pred[i],clf.classes_))
    for i in range(len(y_pred)):
        y_pred[i] = sorted(y_pred[i],key=byVal_key,reverse=True)
        
    y_res = list(range(len(y_pred)))
    for key, val in enumerate(y):
        y_res[key] = 0.0
        for i in range(num_of_cl):
            if val == y_pred[key][i][1]:
                y_res[key] = 1.0
                break
    return np.mean(y_res)

In [7]:
r2 = score_comp(X_test,y_test,clf,num_of_cl=3)
r2

0.7435897435897436

In [10]:
clf.predict(X_test)

array(['italyanskaya', 'russkaya', 'russkaya', 'italyanskaya',
       'gruzinskaya', 'kitayskaya', 'francuzskaya', 'meksikanskaya',
       'sredizemnomorskaya', 'russkaya', 'russkaya', 'yaponskaya',
       'kitayskaya', 'italyanskaya', 'russkaya', 'meksikanskaya',
       'russkaya', 'ispanskaya', 'meksikanskaya', 'kitayskaya',
       'italyanskaya', 'francuzskaya', 'kitayskaya', 'meksikanskaya',
       'kitayskaya', 'russkaya', 'sredizemnomorskaya', 'russkaya',
       'russkaya', 'russkaya', 'russkaya', 'russkaya', 'ispanskaya',
       'russkaya', 'meksikanskaya', 'meksikanskaya', 'kitayskaya',
       'yaponskaya', 'kitayskaya', 'gruzinskaya', 'italyanskaya',
       'yaponskaya', 'sredizemnomorskaya', 'sredizemnomorskaya',
       'kitayskaya', 'kitayskaya', 'russkaya', 'indiyskaya',
       'sredizemnomorskaya', 'francuzskaya', 'francuzskaya', 'ispanskaya',
       'russkaya', 'ispanskaya', 'italyanskaya', 'kitayskaya',
       'indiyskaya', 'kitayskaya', 'russkaya', 'yaponskaya', 'russka

In [82]:
import re
def pred(ingr:str,clf,ingr_l:list):
    arr = re.split(',',ingr)
    for i in range(len(arr)):
        arr[i] = arr[i].capitalize()
    for i in arr:
        print(i)
    test = [[]]
    for i in ingr_l:
        if i in arr:
            test[0].append(1)
        else:
            test[0].append(0)
    return clf.predict(test)[0]


In [83]:
print(pred('яблоко,Финиmки,Мед,Лимонный сок,Ваниль',clf,list(df1.columns)))

Яблоко
Финиmки
Мед
Лимонный сок
Ваниль
francuzskaya


In [37]:
df1.columns

Index(['Блины', 'Баранья грудинка', 'Крутоны', 'Чай аньцзи байча',
       'Грог масала', 'Апельсиновый ароматизатор', 'Финики', 'Соус демигляс',
       'Филе палтуса', 'Черные помидоры',
       ...
       'Мелкие патиссоны', 'Желтый карри порошок', 'Сушеные помидоры',
       'Говяжья пашина', 'Турнепс', 'Мясо', 'Сушеный перец чили',
       'Красный перец чили', 'Шоколадные пряники', 'Яблоки «голден»'],
      dtype='object', length=2397)

In [49]:
s = 'SDASSFAD'
print(s.lower())

TypeError: lower() takes no arguments (1 given)